In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setMaster('spark://master:7077').setAppName('ItemSets')
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/08 00:05:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sc.setLogLevel('ERROR')
sc.version
sc.getConf().getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.app.submitTime', '1670457958325'),
 ('spark.master', 'spark://master:7077'),
 ('spark.driver.host', 'master'),
 ('spark.app.name', 'ItemSets'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '38905'),
 ('spark.app.

Load dataset and preprocessing

In [3]:
fileRDD = (
    sc
    .textFile('data/itemsets.txt', 8) 
    .map(lambda x: x.strip())
)
fileRDD.count()

100000

In [4]:
dataRDD = (
    fileRDD
    .map(lambda x: sorted([int(i) for i in x.split(' ')]))
)

In [5]:
import time

support = 100 # support threshold

# 1. Find the products which are frequently browsed together by using the A-priori algorithm. Find itemsets of size 1, 2 and 3.  
For itemsets size 1, count the number of appearances of each item in the dataset and filter with the support threshold.

In [6]:
start_time = time.time()
itemset1 = (
    dataRDD
    .flatMap(lambda x: x)
    .map(lambda x: ((x,), 1))
    .reduceByKey(lambda a, b: a + b)
    .filter(lambda x: x[1] >= support)
)
print(f'{itemset1.count()} itemsets of size 1')
print(itemset1.take(5))
print(f'Time taken: {time.time() - start_time}s')

797 itemsets of size 1
[((35,), 1984), ((283,), 4082), ((883,), 4902), ((947,), 3690), ((979,), 132)]
Time taken: 4.592349290847778s


For itemsets size 2, we first generate the candidates of size 2 from frequent itemsets size 1 above  
Use `cartersian` to generate the candidates from rdd and filter out duplications  
Then we count the number of appearances of each candidate in the origin dataset and filter by the support threshold

In [7]:
start_time = time.time()
c_2 = itemset1.map(lambda x: (x[0][0]))
c_2 = set(
    c_2
    .cartesian(c_2)
    .filter(lambda x: x[0] < x[1])
    .collect()
)

itemset2 = (
    dataRDD
    .flatMap(
        lambda x: [tuple(sorted((x[i], x[j]))) for i in range(len(x)) for j in range(i+1, len(x))]
    )
    .map(lambda x: (x, 1) if x in c_2 else (x, 0))
    .filter(lambda x: x[1] > 0)
    .reduceByKey(lambda a, b: a + b)
    .filter(lambda x: x[1] >= support)
)

print(f'{itemset2.count()} itemsets of size 2')
print(itemset2.take(5))
print(f'Time taken: {time.time() - start_time}s')

8831 itemsets of size 2
[((52, 538), 247), ((52, 730), 424), ((274, 328), 265), ((274, 368), 156), ((274, 448), 290)]
Time taken: 26.485960483551025s


For itemsets size 3, we generate the candidates of size 3 from the frequent itemsets size 2 using `cartersian`  
Then we count the number of appearances similar the previous block

In [8]:
start_time = time.time()
c_3 = (
    itemset2
    .map(lambda x: x[0])
)
c_3 = set(
    c_3
    .cartesian(c_3)
    .map(lambda x: tuple(sorted(list(set(x[0]) | set(x[1])))))
    .filter(lambda x: len(x) == 3)
    .distinct()
    .collect()
)

itemset3 = (
    dataRDD
    .flatMap(
        lambda x: [tuple(sorted((x[i], x[j], x[k]))) for i in range(len(x)) for j in range(i+1, len(x)) for k in range(j+1, len(x))]
    )
    .filter(lambda x: x[0] < x[1] and x[1] < x[2])
    .map(lambda x: (x, 1) if x in c_3 else (x, 0))
    .filter(lambda x: x[1] > 0)
    .reduceByKey(lambda a, b: a + b)
    .filter(lambda x: x[1] >= support)
)
print(f'{itemset3.count()} itemsets of size 3')
print(itemset3.take(5))
print(f'Time taken: {time.time() - start_time}s')

7130 itemsets of size 3
[((140, 414, 935), 176), ((14, 160, 887), 174), ((161, 558, 774), 224), ((336, 385, 738), 135), ((385, 422, 606), 130)]
Time taken: 194.64307403564453s


# 2. Identify item triples (X, Y, Z) such that the support of {X, Y, Z} is at least 100. For all such triples, compute the confidence scores of the corresponding association rules:  
(X, Y ) → Z,  
(X, Z) → Y,  
(Y, Z) → X.  
Sort the rules in decreasing order of confidence scores and list the top 5 rules.  

The condifence scores of the association rules (X, Y) → Z is: `sup(X, Y, Z) / sup(X, Y)`  
First we find the support for itemsets size 2, store it into a map for easy to access  
Then we use `flatMap` to map each itemset size 3 `(X, Y, Z)` into 3 different association rules `(X, Y ) → Z`, `(X, Z) → Y`, `(Y, Z) → X`.

In [9]:
start_time = time.time()
itemset2_map = {}
for item in itemset2.collect():
    itemset2_map[item[0]] = item[1]

associationRule = (
    itemset3
    .flatMap(
        lambda x: [
            ((x[0][0], x[0][1]) , x[0][2], x[1] / itemset2_map[tuple(sorted((x[0][0], x[0][1])))]),
            ((x[0][1], x[0][2]) , x[0][0], x[1] / itemset2_map[tuple(sorted((x[0][1], x[0][2])))]),
            ((x[0][0], x[0][2]) , x[0][1], x[1] / itemset2_map[tuple(sorted((x[0][0], x[0][2])))])
        ]
    )
    .sortBy(lambda x: -x[2])
)
associationRule.take(5)
print(f'Top 5 rules: {itemset3.take(5)}')
print(f'Time taken: {time.time() - start_time}s')

Top 5 rules: [((120, 124, 205), 243), ((120, 124, 581), 240), ((124, 205, 834), 244), ((124, 581, 834), 244), ((262, 294, 853), 347)]
Time taken: 6.500771760940552s


# 3. Use SON algorithm to identify frequent itemsets with the same value of support.
Use mapPartition to split the dataset into smaller partition and find the frequent itemsets in each partition. Since we divide into 8 partitions, then the support threshold in each partition should be `100/8`. 
Then all the frequent itemsets from all partitions are gather. For each itemset from the partition, we count its number of appearances and compare with the support threshold (`100`)

In [10]:
start_time = time.time()

def localItemset(partition):
    result = []
    support = round(100 / 8)
    data = [x for x in partition]
    
    itemset1 = {}
    for line in data:
        for item in line:
            x = int(item)
            itemset1[x] = itemset1.get(x, 0) + 1
    itemset1 = [(k,) for k, v in itemset1.items() if v >= support]
    
    c_2 = set([
        tuple(sorted((itemset1[i][0], itemset1[j][0])))
        for i in range(len(itemset1))
        for j in range(i+1, len(itemset1))
    ])

    itemset2 = {}
    for line in data:
        for i in range(len(line)):
            for j in range(i+1, len(line)):
                x = tuple(sorted((line[i], line[j])))
                if x in c_2:
                    itemset2[x] = itemset2.get(x, 0) + 1
    itemset2 = [k for k, v in itemset2.items() if v >= support]

    c_3 = []
    for i in range(len(itemset2)):
        for j in range(i+1, len(itemset2)):
            x = set(itemset2[i]) | set(itemset2[j])
            if len(x) == 3:
                c_3.append(tuple(sorted(list(x))))
    c_3 = set(c_3)
    itemset3 = {}
    for line in data:
        for i in range(len(line)):
            for j in range(i+1, len(line)):
                for k in range(j+1, len(line)):
                    x = tuple(sorted((line[i], line[j], line[k])))
                    if x in c_3:
                        itemset3[x] = itemset3.get(x, 0) + 1
    itemset3 = [k for k, v in itemset3.items() if v >= support]
    
    return [itemset1, itemset2, itemset3]

sonResult = dataRDD.mapPartitions(localItemset).collect()

c_1 = set()
c_2 = set()
c_3 = set()
for line in sonResult:
    for x in line:
        if len(x) == 1:
            c_1.add(x)
        if len(x) == 2:
            c_2.add(x)
        if len(x) == 3:
            c_3.add(x)
            
itemset1 = (
    dataRDD
    .flatMap(
        lambda x: x
    )
    .map(lambda x: ((x,), 1) if (x,) in c_1 else ((x,), 0))
    .filter(lambda x: x[1] > 0)
    .reduceByKey(lambda a, b: a + b)
    .filter(lambda x: x[1] >= support)
)
print(f'{itemset1.count()} itemsets of size 1')

itemset2 = (
    dataRDD
    .flatMap(
        lambda x: [(x[i], x[j]) for i in range(len(x)) for j in range(i+1, len(x))]
    )
    .filter(lambda x: x[0] < x[1])
    .map(lambda x: (x, 1) if x in c_2 else (x, 0))
    .filter(lambda x: x[1] > 0)
    .reduceByKey(lambda a, b: a + b)
    .filter(lambda x: x[1] >= support)
)
print(f'{itemset2.count()} itemsets of size 2')

itemset3 = (
    dataRDD
    .flatMap(
        lambda x: [(x[i], x[j], x[k]) for i in range(len(x)) for j in range(i+1, len(x)) for k in range(j+1, len(x))]
    )
    .filter(lambda x: x[0] < x[1] and x[1] < x[2])
    .map(lambda x: (x, 1) if x in c_3 else (x, 0))
    .filter(lambda x: x[1] > 0)
    .reduceByKey(lambda a, b: a + b)
    .filter(lambda x: x[1] >= support)
)
print(f'{itemset3.count()} itemsets of size 3')

print(f'Total time taken: {time.time() - start_time}s')

797 itemsets of size 1


8831 itemsets of size 2


7130 itemsets of size 3
Total time taken: 159.12080717086792s


The result using SON algorithm is similar to the result using Apriori

In [11]:
itemset3.take(5)

[((120, 124, 205), 243),
 ((120, 124, 581), 240),
 ((124, 205, 834), 244),
 ((124, 581, 834), 244),
 ((262, 294, 853), 347)]